In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

product_url = "https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html?piid=1702143331&auctionId=9bdbad2e-226f-4574-890f-49c7ace6183d&adTypeId=1"
payload = {
    "source": "universal",
    "url": product_url,
    "user_agent_type": "desktop_safari",
    "geo_location": "United States",
    "render": "html",
    "browser_instructions": [
        {
            "type": "wait_for_element",
            "selector": {
                "type": "css",
                "value": "div.SFPrice span.oakhm64z_6112"
            },
            "timeout_s": 10
        }
    ]
}

username = "joe528491_EoLxB"
password = "Adej24972497_02"

response = requests.post(
    "https://realtime.oxylabs.io/v1/queries",
    auth=(username, password),
    json=payload,
    timeout=180
)
print(response.status_code)

content = response.json()["results"][0]["content"]
soup = BeautifulSoup(content, "html.parser")

title = soup.find("h1", {"data-hb-id": "Heading"}).text
price = soup.find("div", {"class": "SFPrice"}).find("span", {"class": "oakhm64z_6112"}).text
rating = soup.find("span", {"class": "ProductRatingNumberWithCount-rating"}).text

data = [{
   "Product Title": title,
   "Price": price,
   "Rating": rating,
   "Link": product_url,
}]
df = pd.DataFrame(data)
# df.to_csv("product_data.csv", index=False)
# df.to_json("product_data.json", orient="records")

401


KeyError: 'results'

In [15]:
import requests
from bs4 import BeautifulSoup

# Your Oxylabs API credentials
USERNAME = "joe528491_EoLxB"
PASSWORD = "Adej24972497_01"

# Set up payload
payload = {
    "source": "universal_ecommerce",  # 或是 "universal_ecommerce"，看你要不要加 render
    "url": "https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html",
    "user_agent_type": "desktop_safari",
    "geo_location": "United States",
    "render": "html"  # html代表要完整渲染
}

# Send POST request
response = requests.post(
    "https://realtime.oxylabs.io/v1/queries",
    auth=(USERNAME, PASSWORD),
    json=payload,
    timeout=180
)

# Check response
if response.status_code == 200:
    result = response.json()
    html_content = result["results"][0]["content"]

    # Optional: parse with BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Example: find price
    price_span = soup.find("span", {"data-test-id": "PriceDisplay"})
    if price_span:
        price = price_span.get_text(strip=True)
        print(f"💲 Product Price: {price}")
    else:
        print("⚠️ Price not found.")

else:
    print(f"❌ Request failed with status code {response.status_code}")
    print(response.text)


💲 Product Price: $539.99


In [13]:
print(result)

{'results': [{'content': '', 'created_at': '2025-04-28 00:58:15', 'updated_at': '2025-04-28 00:59:08', 'page': 1, 'url': 'https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html', 'job_id': '7322423874488919042', 'is_render_forced': False, 'status_code': 613}], 'job': {'callback_url': None, 'client_id': 109838, 'context': [{'key': 'force_headers', 'value': False}, {'key': 'force_cookies', 'value': False}, {'key': 'hc_policy', 'value': True}, {'key': 'successful_status_codes', 'value': []}, {'key': 'follow_redirects', 'value': None}, {'key': 'cookies', 'value': []}, {'key': 'headers', 'value': []}, {'key': 'session_id', 'value': None}, {'key': 'http_method', 'value': 'get'}, {'key': 'content', 'value': None}, {'key': 'store_id', 'value': None}, {'key': 'proxy_location', 'value': None}, {'key': 'delivery_location', 'value': None}, {'key': 'fulfillment_type', 'value': None}], 'created_at': '2025-04-28 00:58:15', 'dom

In [14]:
print(response.json())


{'results': [{'content': '', 'created_at': '2025-04-28 00:58:15', 'updated_at': '2025-04-28 00:59:08', 'page': 1, 'url': 'https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html', 'job_id': '7322423874488919042', 'is_render_forced': False, 'status_code': 613}], 'job': {'callback_url': None, 'client_id': 109838, 'context': [{'key': 'force_headers', 'value': False}, {'key': 'force_cookies', 'value': False}, {'key': 'hc_policy', 'value': True}, {'key': 'successful_status_codes', 'value': []}, {'key': 'follow_redirects', 'value': None}, {'key': 'cookies', 'value': []}, {'key': 'headers', 'value': []}, {'key': 'session_id', 'value': None}, {'key': 'http_method', 'value': 'get'}, {'key': 'content', 'value': None}, {'key': 'store_id', 'value': None}, {'key': 'proxy_location', 'value': None}, {'key': 'delivery_location', 'value': None}, {'key': 'fulfillment_type', 'value': None}], 'created_at': '2025-04-28 00:58:15', 'dom